<a href="https://colab.research.google.com/github/charan-chaitanya/Sentimental_Analysis_Major_Project/blob/main/charanmajorproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 11.3MB/s 


In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs = SentimentIntensityAnalyzer()

In [3]:
from bs4 import BeautifulSoup
import numpy as np

In [4]:
import pandas as pd
df=pd.read_csv('https://raw.githubusercontent.com/charan-chaitanya/Datasets/main/Test.csv')
df

,text,label
0,I always wrote this series off as being a comp...,0
1,1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...,0
2,This movie was so poorly written and directed ...,0
3,The most interesting thing about Miryang (Secr...,1
4,"when i first read about ""berlin am meer"" i did...",0
...,...,...
4995,This is the kind of picture John Lassiter woul...,1
4996,A MUST SEE! I saw WHIPPED at a press screening...,1
4997,NBC should be ashamed. I wouldn't allow my chi...,0
4998,This movie is a clumsy mishmash of various gho...,0


In [5]:
import nltk
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
#remove html tags
def html_tags(text):
  soup = BeautifulSoup(text,"html.parser")
  new_text = soup.get_text()
  return new_text

In [7]:
# expand contractions
!pip install contractions
import contractions
def con(text):
  expand = contractions.fix(text)
  return expand

     |████████████████████████████████| 266kB 11.3MB/s 
     |████████████████████████████████| 327kB 10.8MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.1-cp37-cp37m-linux_x86_64.whl size=85265 sha256=dfb0adad5f458aabfa4e2aef84367b313b1f8adb944c9f23a6762e02264e002a
  Stored in directory: /root/.cache/pip/wheels/e4/ab/f7/cb39270df8f6126f3dd4c33d302357167086db460968cfc80c
Successfully built pyahocorasick


In [8]:
#special characters
import re
def remove_sp(text):
  pattern = r'[^A-Za-z0-9\s]'
  text = re.sub(pattern,'',text)
  return text

In [9]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()

In [10]:
def remove_stopwords(text):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  filtered_tokens = [token for token in tokens if token not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)
  return filtered_text

In [11]:
df.text = df.text.apply(lambda x:x.lower())
df.text = df.text.apply(html_tags)
df.text = df.text.apply(con)
df.text = df.text.apply(remove_sp)
df.text = df.text.apply(remove_stopwords)

df

,text,label
0,always wrote series complete stinkfest jim bel...,0
1,1st watched 1272002 3 10dirsteve purcell typic...,0
2,movie poorly written directed fell asleep 30 m...,0
3,interesting thing miryang secret sunshine acto...,1
4,first read berlin meer not expect much thought...,0
...,...,...
4995,kind picture john lassiter would making today ...,1
4996,must see saw whipped press screening hilarious...,1
4997,nbc ashamed would not allow children see defin...,0
4998,movie clumsy mishmash various ghoststory suspe...,0


In [12]:
df['compound'] = df['text'].apply(lambda x: vs.polarity_scores(x)['compound'])

In [13]:
df

,text,label,compound
0,always wrote series complete stinkfest jim bel...,0,0.9876
1,1st watched 1272002 3 10dirsteve purcell typic...,0,0.5624
2,movie poorly written directed fell asleep 30 m...,0,-0.8829
3,interesting thing miryang secret sunshine acto...,1,-0.9813
4,first read berlin meer not expect much thought...,0,-0.4677
...,...,...,...
4995,kind picture john lassiter would making today ...,1,0.9425
4996,must see saw whipped press screening hilarious...,1,0.9880
4997,nbc ashamed would not allow children see defin...,0,-0.5213
4998,movie clumsy mishmash various ghoststory suspe...,0,-0.9543


In [14]:
def compoundcondition(row):
  if(row['compound'] > 0.05):
    return 1
  if(row['compound'] < -0.05):
    return -1
  if (row['compound'] > -0.05 and row['compound'] < 0.05):
    return 0

In [15]:
df['new_column'] = df.apply(lambda row: compoundcondition(row), axis=1)
df

,text,label,compound,new_column
0,always wrote series complete stinkfest jim bel...,0,0.9876,1
1,1st watched 1272002 3 10dirsteve purcell typic...,0,0.5624,1
2,movie poorly written directed fell asleep 30 m...,0,-0.8829,-1
3,interesting thing miryang secret sunshine acto...,1,-0.9813,-1
4,first read berlin meer not expect much thought...,0,-0.4677,-1
...,...,...,...,...
4995,kind picture john lassiter would making today ...,1,0.9425,1
4996,must see saw whipped press screening hilarious...,1,0.9880,1
4997,nbc ashamed would not allow children see defin...,0,-0.5213,-1
4998,movie clumsy mishmash various ghoststory suspe...,0,-0.9543,-1


In [16]:
df.shape

(5000, 4)

In [17]:
x = df.iloc[:,0].values
y = df.iloc[:,3].values

In [18]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 0)

In [19]:
x_train.shape

(3500,)

In [20]:
np.unique(y_train,return_counts=True)

(array([-1,  0,  1]), array([1178,   40, 2282]))

In [21]:
np.unique(y_test,return_counts=True)

(array([-1,  0,  1]), array([486,  21, 993]))

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

In [23]:
text_model = Pipeline([('tfidf',TfidfVectorizer()),('model',SVC())]) 

In [24]:
text_model.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('model',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', d

In [25]:
y_pred = text_model.predict(x_test)
y_pred

array([-1,  1,  1, ..., -1,  1,  1])

In [26]:
np.unique(y_pred,return_counts=True)

(array([-1,  1]), array([ 227, 1273]))

In [27]:
np.unique(y_test,return_counts=True)

(array([-1,  0,  1]), array([486,  21, 993]))

In [28]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
accuracy_score(y_pred,y_test)*100

75.86666666666667

In [29]:
confusion_matrix(y_pred,y_test)

array([[185,   2,  40],
       [  0,   0,   0],
       [301,  19, 953]])

In [30]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

          -1       0.38      0.81      0.52       227
           0       0.00      0.00      0.00         0
           1       0.96      0.75      0.84      1273

    accuracy                           0.76      1500
   macro avg       0.45      0.52      0.45      1500
weighted avg       0.87      0.76      0.79      1500



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
!pip install streamlit
!pip install pyngrok==4.1.1
!pip install contractions
!pip install vaderSentiment
from pyngrok import ngrok

     |████████████████████████████████| 7.0MB 12.7MB/s 
     |████████████████████████████████| 112kB 48.6MB/s 
     |████████████████████████████████| 4.6MB 46.9MB/s 
     |████████████████████████████████| 81kB 6.4MB/s 
     |████████████████████████████████| 163kB 47.8MB/s 
     |████████████████████████████████| 122kB 54.6MB/s 
     |████████████████████████████████| 71kB 4.8MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp37-none-any.whl size=13448 sha256=ccdc58e1160429080166bfc9fc38f80e2c87a9d7d466d1cbfeb082cb560ce905
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.0 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


  Created wheel for pyngrok: filename=pyngrok-4.1.1-cp37-none-any.whl size=15971 sha256=1af2e7b9a530050002020699794a294cd52f662354084d40124a478fe6d2f5f3
  Stored in directory: /root/.cache/pip/wheels/97/71/0d/1695f7c8815c0beb3b5d9b35d6eec9243c87e6070fbe3977fa
Successfully built pyngrok


In [32]:
%%writefile app.py
import streamlit as st
st.title('MAJOR PROJECT')
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/charan-chaitanya/Datasets/main/Test.csv')
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import os
import nltk
nltk.download('stopwords')
stopwords_list = nltk.corpus.stopwords.words('english')
stopwords_list.remove('no')
stopwords_list.remove('not')
def html_tag(text):
  soup = BeautifulSoup(text,"html.parser")
  new_text = soup.get_text()
  return new_text
import contractions
def con(text):
  expand = contractions.fix(text)
  return expand
import re 
def remove_sp(text):
  pattern = r'[^A-Za-z\s]'
  text = re.sub(pattern,'',text)
  return text
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
def remove_stopwords(text):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  filter_tokens = [token for token in tokens if token not in stopwords_list]
  filter_tokens =' '.join(filter_tokens)
  return filter_tokens
df.text = df.text.apply(lambda x:x.lower())
df.text = df.text.apply(html_tag)
df.text = df.text.apply(con)
df.text = df.text.apply(remove_sp)
df.text = df.text.apply(remove_stopwords)
from vaderSentiment.vaderSentiment import  SentimentIntensityAnalyzer
vs = SentimentIntensityAnalyzer()
df['compound'] = df['text'].apply(lambda x: vs.polarity_scores(x)['compound'])
def compoundcondition(row):
  if (row['compound'] > 0.05):
    return 1
  if (row['compound'] < -0.05):
    return -1
  if (row['compound'] > -0.05 and row['compound'] < 0.05):
    return 0
df['new_column'] = df.apply(lambda row : compoundcondition(row),axis =1)
x = df.iloc[:,0].values
y = df.iloc[:,3].values
select = st.text_input('Enter your message')
if st.button('PRED'):
  from sklearn.pipeline import Pipeline
  from sklearn.feature_extraction.text import TfidfVectorizer
  from sklearn.svm import SVC
  text_model = Pipeline([('tfidf',TfidfVectorizer()),('model',SVC())]) 
  text_model.fit(x,y)
  op = text_model.predict([select])
  if ( op == 1 ): 
    result = "POSITIVE 😃"
  elif (op ==0 ):
    result = "NEUTRAL 😐"
  elif ( op == -1):
    result = "NEGATIVE 😔"
  st.title(result)

Writing app.py


In [33]:
!nohup streamlit run app.py &
url = ngrok.connect(port='8501')
url

nohup: appending output to 'nohup.out'


'http://0564919b1638.ngrok.io'